In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [14, 5]
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/drive')
BASE_PATH = '/drive/My Drive/02_coding/04_notebooks/01_modulos_challenge'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive


# Start

- The first step in analyzing any dataset is to visualize and check out the behaviour of the variables first.
- As I understand the task in the 3 datasets we are looking at different kind of anomalies in the data. I guess the goal would be to have a model for each dataset predicting if there was an anomaly or not.
- A problem is the metric to decide if we have a good performance. Since we are in a unsupervised setting and we do not actually know what an anomaly is, we must resort to a lot of qualitative analysis and look at samples and the result. 
- Therefore it is a good start to take out a few of the samples of each dataset and look at them.

## Train_Dataset_1.npy

In [8]:
df = pd.DataFrame(np.load(BASE_PATH + 'data/Train_Dataset_1.npy'))
random_indices = np.random.randint(0, 1999, (10,))

def plot_simple(data):
    sns.lineplot(x=range(1200), y=data)
    plt.ylim(0,1.3)
    plt.show()

for x in random_indices:
    print("Checking out row {}".format(x))
    plot_simple(np.array(df.loc[[x]]).squeeze())

Checking out row 281


AttributeError: ignored

- We can instantly see that there are two different kind of modes:
    - In the first mode the variable we observe oscillates around 1 +/- $\epsilon$, with different errors but the mean is definitely around 1.
    - In the second mode we can see clear outbreaks, the mean and error of the oscillation change drastically. 
- Without any input from the client and without knowing what variables we are looking at it is difficult to state with certainty what is an anomaly and what not.
- So we need to make an assumption here, to know what we are looking for. However the results have to be understood as a result of this assumption, if we find out from the client or another way that this assumption is incorrect we need start back here with the new assumption.
- We will assume for the following the the outbreaks we observe in the second modes are the anomalies we are looking for.
- In cases like these where we can see that we have two different means and errors the next step would be to plot histograms together with the values of the values and see if we maybe can see the two different distributions.

In [0]:
def plot_with_histo(data):
    plt.figure()
    plt.subplot(1,2,1)
    sns.lineplot(x=range(1200), y=data)
    plt.ylim(0,1.3)
    plt.subplot(1,2,2)
    sns.distplot(data)
    plt.xlim(0, 1.3)
    plt.ylim(0, 25)
    plt.show()

random_indices = np.random.randint(0, 1999, (10,))
for x in random_indices:
    print("Checking out row {}".format(x))
    plot_with_histo(np.array(df.loc[[x]]).squeeze())

- In these plots we can quite clearly see that when we have an anomaly the distribution of the datapoints shows two modes.
- From the visualizations we can conclude that we can approximate these modes using gaussians.
- So now we have a strategy for a baseline predictor:
    - As a first step we can cluster the datapoints of each sample. Since we use gaussians to approximate the datapoints we can use gaussian mixture models to fit the datapoints.
    - As a second step we need to find out which of the modes is the anomaly. Any datapoint falling into this mode can be classified as an anomaly.
- This probabilistic approach is also good to combat false positives and negatives. The result of the clustering will be two probability distributions. For each datapoint we can then compute the probability of lying in a particular cluster. This allows us to use different strategies in classifying the datapoints, this is something that can be optimized also during operation.
- We are only looking to fit 1 or 2 Gaussian Modes per sample, therefore we can make an exhaustive search and visualize the results.
- We can choose the better fit using the BIC criterion.
- Now lets do that and look at the results we get.

In [0]:
from sklearn.mixture import GaussianMixture

def plot_classified(sample, labels):
    plt.figure()
    
    data_0 = []
    data_1 = []
    for x in list(range(len(sample))):
        value = sample[x]
        if labels[x] == 0:
            data_0.append((x, value))
        else:
            data_1.append((x, value))
    
    if len(data_0) > 1:
        x_0, y_0 = zip(*data_0)
        print('Num datapoints in class 0: {}'.format(len(x_0)))
    if len(data_1) > 1:
        x_1, y_1 = zip(*data_1)
        print('Num datapoints in class 1: {}'.format(len(x_1)))
    
    # Simple plot of series with predicted labels
    plt.subplot(1,2,1)
    if len(data_0) > 1:
        sns.lineplot(x=x_0, y=y_0)
    if len(data_1) > 1:
        sns.lineplot(x=x_1, y=y_1)
    plt.ylim(-2,2.3)
    
    # Histogram of series with predicted labels
    plt.subplot(1,2,2)
    if len(data_0) > 1:
        sns.distplot(y_0)
    if len(data_1) > 1:
        sns.distplot(y_1)
    plt.xlim(-2,2.3)
    plt.ylim(0, 25)
    plt.show()

def fit(sample, n_components):
    clf = GaussianMixture(n_components=n_components, covariance_type='full')
    labels = clf.fit_predict(sample.reshape(-1, 1))
    bic = clf.bic(sample.reshape(-1, 1))
    return bic, labels

random_indices = np.random.randint(0, 1999, (10,))
for x in random_indices:
    print('================================================================================')
    print("Processing row {}".format(x))
    sample = np.asarray(df.loc[[x]].transpose()).squeeze()
    bic_1, labels_1 = fit(sample, n_components=1)
    bic_2, labels_2 = fit(sample, n_components=2)
    
    if bic_1 < bic_2:
        plot_classified(sample, labels_1)
    else:
        plot_classified(sample, labels_2)

- We can quickly see that we are not too bad in classifying the data points.
- Problems arise when the two distributions overlap. Then the model can not reliably detect the cluster.
- Since we are dealing with sequential data we can use this information. We need to do some feature engineering. For example we could try to predict the label of a datapoint from the past few datapoints. Like this we would kind of regularize the noisy results. We can extract these features and apply a simple SVM approach and look at what we get. We have to try out different window sizes.
- In the following we will only display cases with two modes, since these are the interesting ones.

In [0]:
from sklearn.svm import SVC

def create_features(series, window_size):
    x = []
    y = []
    for idx, label in enumerate(series[window_size:]):
        x.append(series[idx:idx+window_size-1])
        y.append(label)
    return np.asarray(x), np.asarray(y)

def optimize_labels(labels):
    window_size = 10
    X, y = create_features(labels, window_size)
    if len(set(y)) == 1:
        return labels
    clf = SVC()
    clf.fit(X, y)
    labels_new = clf.predict(X)
    return np.append(labels[:window_size], labels_new)
        
random_indices = np.random.randint(0, 1999, (30,))
for x in random_indices:
    sample = np.asarray(df.loc[[x]].transpose()).squeeze()
    bic_1, labels_1 = fit(sample, n_components=1)
    bic_2, labels_2 = fit(sample, n_components=2)
    if bic_1 < bic_2:
        pass
    else:
        print('================================================================================')
        print("Processing row {}".format(x))
        print("Optimized Labels")
        plot_classified(sample, optimize_labels(labels_2))
        print("Not optimized labels")
        plot_classified(sample, labels_2)

- When we compare the results of the optimized labels with the not optimized labels we can see that we can really improve the results by adding a time series classifier on top of the original model

- However there are also problems. Sometimes the label optimizer maps all labels to 1 class. This happens mostly when we have two distributions with the same mean, but different variance.
- When optimizing the labels we already know that we are dealing with 2 classes. Therefore we do not want to map everything to one class. 

- Until now we did not apply any preprocessing to the data. Essentially we are not interested in what the exact distribution of the anomaly is, we just want to know when it happens. Therefore we can use transformations to try amplify the difference between the two distributions.

- First we try a simple method. We keep a running window of means and then we amplify the difference between the mean and the value. What whe do in essence is we stretch the variance of the gaussians. The bigger the variance the bigger the increase, therefore the differences become more apparent.

In [0]:
def preprocess(series):
    window_size = 10
    preprocessed_series = []
    for idx, val in enumerate(series[window_size:]):
        mean = np.mean(series[idx:idx+window_size-1])
        preprocessed_series.append(mean+2*(val-mean))
    return np.concatenate((series[:window_size], preprocessed_series))
    
    

random_indices = np.random.randint(0, 1999, (10,))
for x in random_indices:
    sample = np.asarray(df.loc[[x]].transpose()).squeeze()
    preprocessed_data = preprocess(sample)
    print("Raw vs. Preprocessed data sample {}".format(x))
    plt.figure()
    plt.subplot(1,2,1)
    sns.lineplot(x=range(len(sample)), y=sample)
    plt.ylim(-2,2.3)
    plt.subplot(1,2,2)
    sns.lineplot(x=range(len(sample)), y=preprocessed_data)
    plt.ylim(-2,2.3)
    plt.show()

- As we can see by looking at some samples we can see that we can achieve the desired effect. The next step is to apply our model to the preprocessed data.

In [0]:
random_indices = np.random.randint(0, 1999, (30,))
for x in random_indices:
    sample = np.asarray(df.loc[[x]].transpose()).squeeze()
    preprocessed_data = preprocess(sample)
    bic_1, labels_1 = fit(sample, n_components=1)
    bic_2, labels_2 = fit(sample, n_components=2)
    if bic_1 < bic_2:
        pass
    else:
        print('================================================================================')
        print("Processing row {}".format(x))
        print("Preprocessed + optimized Labels")
        plot_classified(preprocessed_data, optimize_labels(labels_2))
        print("Not preprocessed + optimized labels")
        plot_classified(sample, optimize_labels(labels_2))

- As we can see in the results we can achieve a better separation by preprocessing the data. 
- Even if the distributions overlap, we can distinguish two different gaussians since the difference in variance is amplified.
- As we have seen during the experiments the behvaviour which we have interpreted as "normal" have a mean close to 1. At least when looking at samples we can see the the anomalous behaviour has a mean that is further from 1 than the normal behaviour.
- We can use this assumption to preprocess the data even more, we can amplify the difference from 1 to achieve a clearer separation of means:

In [0]:
def preprocess_stage_2(series):
    assert(len(series) == 1200)
    series = preprocess(series)
    window_size = 10
    preprocessed_series = []
    for idx, val in enumerate(series[window_size:]):
        mean = np.mean(series[idx:idx+window_size-1])
        preprocessed_series.append(val+2*(mean-1))
    return np.concatenate((series[:window_size], preprocessed_series))

random_indices = np.random.randint(0, 1999, (10,))
for x in random_indices:
    sample = np.asarray(df.loc[[x]].transpose()).squeeze()
    preprocessed_data = preprocess_stage_2(sample)
    print("Raw vs. Preprocessed data sample {}".format(x))
    plt.figure()
    plt.subplot(1,2,1)
    sns.lineplot(x=range(len(sample)), y=sample)
    plt.ylim(-2,2.3)
    plt.subplot(1,2,2)
    sns.lineplot(x=range(len(sample)), y=preprocessed_data)
    plt.ylim(-2,2.3)
    plt.show()

- We can see that we can achieve the desired result. Now lets look at our predictions using the second stage preprocessing:

In [0]:
random_indices = np.random.randint(0, 1999, (30,))
for x in random_indices:
    sample = np.asarray(df.loc[[x]].transpose()).squeeze()
    preprocessed_data = preprocess_stage_2(sample)
    bic_1, labels_1 = fit(sample, n_components=1)
    bic_2, labels_2 = fit(sample, n_components=2)
    if bic_1 < bic_2:
        pass
    else:
        print('================================================================================')
        print("Processing row {}".format(x))
        print("Preprocessed Stage 2 + optimized Labels")
        plot_classified(preprocessed_data, optimize_labels(labels_2))
        print("Not preprocessed + optimized labels")
        plot_classified(sample, optimize_labels(labels_2))

- As we can see we have very few errors left. I think it would take a lot of effort to produce better results. At least one can build a prototype with this.
- Therefore we can look at the results in the other datasets:

## Train_Dataset_2.npy

In [0]:
df = pd.DataFrame(np.load('data/Train_Dataset_2.npy'))

random_indices = np.random.randint(0, 1999, (30,))
for x in random_indices:
    sample = np.asarray(df.loc[[x]].transpose()).squeeze()
    preprocessed_data = preprocess_stage_2(sample)
    bic_1, labels_1 = fit(sample, n_components=1)
    bic_2, labels_2 = fit(sample, n_components=2)
    if bic_1 < bic_2:
        pass
    else:
        print('================================================================================')
        print("Processing row {}".format(x))
        print("Preprocessed Stage 2 + optimized Labels")
        plot_classified(preprocessed_data, optimize_labels(labels_2))
        print("Not preprocessed + optimized labels")
        plot_classified(sample, labels_2)

## Train_Dataset_3.npy

In [0]:
df = pd.DataFrame(np.load('data/Train_Dataset_2.npy'))

random_indices = np.random.randint(0, 1999, (30,))
for x in random_indices:
    sample = np.asarray(df.loc[[x]].transpose()).squeeze()
    preprocessed_data = preprocess_stage_2(sample)
    bic_1, labels_1 = fit(sample, n_components=1)
    bic_2, labels_2 = fit(sample, n_components=2)
    if bic_1 < bic_2:
        pass
    else:
        print('================================================================================')
        print("Processing row {}".format(x))
        print("Preprocessed Stage 2 + optimized Labels")
        plot_classified(preprocessed_data, optimize_labels(labels_2))
        print("Not preprocessed + optimized labels")
        plot_classified(sample, labels_2)

- When we look at the results, we can see that we still make error as the difficulty of detecting the anomaly progresses. However I think using these predictions and clever heuristics we can find out by talking to the client or looking at the real operation we could achieve a sound solution, at least a solution that can be tested in the field and optimized after getting some more about the system performance.
- The model still fails when whe have a visually interpretable anomaly but the mean and variance of the distribution is really close. That means the GMM detects 2 modes, but they are so interleaved we can not clearly separate them. Here we would need to know if these modes that are so similar are actually anomalies or might be perturbations of the system or measuring device. Therefore it does not make sense to invest a lot of energy before testing this.
- Now the final heuristic we apply is based on the last optimization. When getting two different modes from the model we choose the values that have a mean further from 1 to be the anomaly.
- Of course we can still do a lot of optimization of hyperparameters etc.

Now we can make the predictions for each dataset:

In [0]:
def fit_and_choose(sample):
    bic_1, labels_1 = fit(sample, n_components=1)
    bic_2, labels_2 = fit(sample, n_components=2)
    if bic_1 < bic_2:
        return labels_1
    return labels_2

def translate_labels(sample_labels):
    mean_0 = np.mean(list(map(lambda x: x[0], filter(lambda x: x[1] == 0, zip(*sample_labels)))))
    mean_1 = np.mean(list(map(lambda x: x[0], filter(lambda x: x[1] == 1, zip(*sample_labels)))))
    if abs(mean_0-1) > abs(mean_1-1):
        result = np.invert(sample_labels[1].astype(np.bool))
        return result
    return sample_labels[1]
    
def make_predictions(filename):
    X = np.load('data/' + filename)
    preprocessed_X = np.apply_along_axis(preprocess_stage_2, 1, X)
    predicted_X = np.apply_along_axis(fit_and_choose, 1, preprocessed_X)
    optimized_X = np.apply_along_axis(optimize_labels, 1, predicted_X)
    preprocessed_optimized_X = zip(preprocessed_X, optimized_X)
    translated_X = list(map(translate_labels, preprocessed_optimized_X))
    np.save('predictions/' + filename, translated_X)
    

make_predictions('Eval_Dataset_1.npy')
make_predictions('Eval_Dataset_2.npy')
make_predictions('Eval_Dataset_3.npy')